In [1]:
import kagglehub
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Download latest version
data_path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")
data_path = os.path.join(data_path, 'chest_xray')

100%|██████████| 2.29G/2.29G [01:45<00:00, 23.5MB/s]

Extracting files...


In [34]:
import os
import random
import shutil
from math import ceil

def organize_dataset(original_dataset, new_dataset, split_ratios=(0.8, 0.1, 0.1)):
    """
    Combine, shuffle, and split dataset into train, val, and test sets.

    :param original_dataset: Path to the original dataset folder
    :param new_dataset: Path to the new dataset folder
    :param split_ratios: Tuple indicating the train, val, test split ratios
    """
    assert sum(split_ratios) == 1.0, "Split ratios must sum to 1.0"
    
    # Combine all data into a single list
    all_data = []
    class_names = os.listdir(os.path.join(original_dataset, 'train'))  # Assuming all sets have the same classes

    for class_name in class_names:
        for folder in ['train', 'val', 'test']:
            class_folder = os.path.join(original_dataset, folder, class_name)
            if os.path.exists(class_folder):
                images = [(os.path.join(class_folder, img), class_name) for img in os.listdir(class_folder)]
                all_data.extend(images)

    # Shuffle the combined data
    random.shuffle(all_data)

    # Calculate split sizes
    total_images = len(all_data)
    train_size = ceil(split_ratios[0] * total_images)
    val_size = ceil(split_ratios[1] * total_images)

    train_data = all_data[:train_size]
    val_data = all_data[train_size:train_size + val_size]
    test_data = all_data[train_size + val_size:]

    # Create new folder structure
    for split, split_data in zip(['train', 'val', 'test'], [train_data, val_data, test_data]):
        for img_path, class_name in split_data:
            split_class_dir = os.path.join(new_dataset, split, class_name)
            os.makedirs(split_class_dir, exist_ok=True)
            shutil.copy(img_path, os.path.join(split_class_dir, os.path.basename(img_path)))

    print(f"Dataset organized successfully into {new_dataset}")
    print(f"Train: {len(train_data)} images, Val: {len(val_data)} images, Test: {len(test_data)} images")



In [35]:
new_dataset = "../chest_xray"
organize_dataset(data_path, new_dataset)

Dataset organized successfully into ../chest_xray
Train: 4685 images, Val: 586 images, Test: 585 images


In [10]:
import wandb
wandb.login()

wandb: Currently logged in as: vmazeeva (vmazeeva-carnegie-mellon-university). Use `wandb login --relogin` to force relogin


True

In [6]:
os.environ["WANDB_SILENT"] = "true"

In [7]:
! git clone https://github.com/verammaz/MedImageClassXAI

Cloning into 'MedImageClassXAI'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 46 (delta 12), reused 44 (delta 10), pack-reused 0 (from 0)
Receiving objects: 100% (46/46), 46.28 KiB | 15.43 MiB/s, done.
Resolving deltas: 100% (12/12), done.


In [8]:
os.chdir('/content/MedImageClassXAI')

In [19]:
! python cnn.py -data_dir {data_path} -v -n_epochs 4

Number of classes    :  2
Shape of image       :  torch.Size([1, 256, 256])
Train batches        :  82
No. of train images  :  5216
No. of valid images  :  16
Labels               :  {'NORMAL': 0, 'PNEUMONIA': 1}
Image stats (mean, std):  tensor(0.5519) tensor(0.2131)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 256]             640
       BatchNorm2d-2         [-1, 64, 256, 256]             128
              ReLU-3         [-1, 64, 256, 256]               0
         MaxPool2d-4         [-1, 64, 128, 128]               0
            Conv2d-5        [-1, 128, 128, 128]          73,856
       BatchNorm2d-6        [-1, 128, 128, 128]             256
              ReLU-7        [-1, 128, 128, 128]               0
         MaxPool2d-8          [-1, 128, 64, 64]               0
            Conv2d-9          [-1, 256, 64, 64]         295,168
      BatchNorm2d-10      

In [33]:
! python resnet.py -data_dir {data_path} -n_epochs 1 -batch_size 16

python(99044) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


/opt/anaconda3/envs/ml_ai/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/ml_ai/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Device: mps

Epoch 1
----------------------
Train Loss: 0.26654359522315624, Train Accuracy: 88.47776073619632              
Val Loss: 0.6563367247581482, Val Accuracy: 62.5                                
Test Loss: 0.5717127575324132, Test Accuracy: 71.47435897435898                 
Traceback (most recent call last):
  File "/Users/veramazeev